In [1]:
import pandas as pd
import numpy as np
import openpyxl
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import torch
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

In [3]:
data_root = '/root/ML-Regression/Competition/data/'
train_df = pd.read_csv(data_root + 'train.csv')
test_df = pd.read_csv(data_root + 'test.csv')

/tmp/ipykernel_278359/408154191.py:2: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(data_root + 'train.csv')


In [4]:
train_df.head(3)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,123500
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,91500


In [5]:
# 시군구를 시, 구, 동으로 나누기
train_df['시'] = train_df['시군구'].str.split(' ').str[0]
train_df['구'] = train_df['시군구'].str.split(' ').str[1]
train_df['동'] = train_df['시군구'].str.split(' ').str[2]

In [6]:
train_df.head(3)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,시,구,동
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000,서울특별시,강남구,개포동
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,123500,서울특별시,강남구,개포동
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,91500,서울특별시,강남구,개포동


In [ ]:
# 계약년월을 년, 월로 나누기
train_df['계약년'] = train_df['계약년월'].astype(str).str[:4]
train_df['계약월'] = train_df['계약년월'].astype(str).str[4:]
train_df.drop('계약년월', axis=1, inplace=True)

In [9]:
train_df.head(3)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약일,층,건축년도,도로명,...,관리비 업로드,좌표X,좌표Y,단지신청일,target,시,구,동,계약년,계약월
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,8,3,1987,언주로 3,...,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000,서울특별시,강남구,개포동,2017,12
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,22,4,1987,언주로 3,...,N,127.05721,37.476763,2022-11-17 10:19:06.0,123500,서울특별시,강남구,개포동,2017,12
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,28,5,1987,언주로 3,...,N,127.05721,37.476763,2022-11-17 10:19:06.0,91500,서울특별시,강남구,개포동,2017,12


In [10]:
train_df['계약년'] = train_df['계약년'].astype(str)
train_df['계약월'] = train_df['계약월'].astype(str)
train_df['계약일'] = train_df['계약일'].astype(str)

# 나눈 계약년, 계약월, 계약일을 2021-01-01 형태로 합치기(datetime 형태로 변환)
train_df['계약년월일'] = train_df['계약년'] + '-' + train_df['계약월'] + '-' + train_df['계약일']
train_df['계약년월일'] = pd.to_datetime(train_df['계약년월일'])
train_df.head(3)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약일,층,건축년도,도로명,...,좌표X,좌표Y,단지신청일,target,시,구,동,계약년,계약월,계약년월일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,8,3,1987,언주로 3,...,127.05721,37.476763,2022-11-17 10:19:06.0,124000,서울특별시,강남구,개포동,2017,12,2017-12-08
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,22,4,1987,언주로 3,...,127.05721,37.476763,2022-11-17 10:19:06.0,123500,서울특별시,강남구,개포동,2017,12,2017-12-22
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,28,5,1987,언주로 3,...,127.05721,37.476763,2022-11-17 10:19:06.0,91500,서울특별시,강남구,개포동,2017,12,2017-12-28


In [12]:
train_df['계약년월일'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1118822 entries, 0 to 1118821
Series name: 계약년월일
Non-Null Count    Dtype         
--------------    -----         
1118822 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 8.5 MB


In [14]:
# 계약년월일과 target값인 거래금액만 남기고 모두 제거
train_df = train_df[['계약년월일', 'target']]
train_df.head(3)

,계약년월일,target
0,2017-12-08,124000
1,2017-12-22,123500
2,2017-12-28,91500
